In [1]:
import os, json, time, logging
from datetime import datetime, timedelta
import numpy as np
from scipy.optimize import differential_evolution
from scipy.stats.qmc import Halton

from qiskit import QuantumCircuit
from qiskit.circuit import Parameter
from qiskit_aer import AerSimulator
from qiskit_ibm_runtime import Estimator
from qiskit.quantum_info import SparsePauliOp

from susy_qm import calculate_Hamiltonian2

In [2]:
import qiskit
import qiskit_aer

print("Qiskit version:", qiskit.__version__)
print("Qiskit Aer version:", qiskit_aer.__version__)


Qiskit version: 1.2.4
Qiskit Aer version: 0.16.1


In [5]:
potential = 'QHO'
cutoff = 8
H = calculate_Hamiltonian2(cutoff, potential)
eigenvalues = np.sort(np.linalg.eig(H)[0])[:4]
min_eigenvalue = np.min(eigenvalues)

num_qubits = int(1 + np.log2(cutoff))
observable = SparsePauliOp.from_operator(H)

In [6]:
num_shots = 1024
backend = AerSimulator()
estimator = Estimator(backend, options={"default_shots": num_shots})

In [11]:
num_params = 1
param_objs = [Parameter(f"θ{i}") for i in range(num_params)]

qc = QuantumCircuit(num_qubits)
qc.x(0)
qc.ry(param_objs[0], 3)
#qc.ry(param_objs[1], 2)
#qc.ry(param_objs[2], 1)
#qc.ry(param_objs[3], 0)

In [110]:
num_params = 1
param_objs = [Parameter(f"θ{i}") for i in range(num_params)]

qc = QuantumCircuit(num_qubits)
qc.ry(param_objs[0], 2)   
qc.cry(param_objs[1], 2, 1)   
qc.ry(param_objs[2], 0)     
qc.ry(param_objs[3], 1)    

In [12]:
qc.draw()

┌───┐   
q_0: ──┤ X ├───
       └───┘   
q_1: ──────────
               
q_2: ──────────
     ┌────────┐
q_3: ┤ Ry(θ0) ├
     └────────┘

In [13]:
def cost_function(params):

    param_dict = dict(zip(param_objs, params))
    bound = qc.assign_parameters(param_dict, inplace=False)

    job = estimator.run([(bound, observable)])
    energy = job.result()[0].data.evs
    #print('estimator expectation val:', energy)

    return energy

In [14]:
bounds = [(0, 2 * np.pi) for _ in range(num_params)]
max_iter = 10000
strategy = "randtobest1bin"
tol = 1e-3
abs_tol = 1e-3
popsize = 20

seed = (os.getpid() * int(time.time())) % 123456789
run_start = datetime.now()

halton_sampler = Halton(d=num_params, seed=seed)
scaled_samples = 2 * np.pi * halton_sampler.random(n=popsize)


result = differential_evolution(
    cost_function,
    bounds=bounds,
    maxiter=max_iter,
    tol=tol,
    atol=abs_tol,
    strategy=strategy,
    popsize=popsize,
    init=scaled_samples,
    seed=seed,
    workers=1
    )

In [15]:
result.fun

np.float64(2.220446049250313e-16)